<a href="https://colab.research.google.com/github/nayun12/Tobigs_17/blob/main/week3_DT_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DT Assignment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Loading

In [4]:
import pandas as pd 
import numpy as np

In [6]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_gini(df, label):
    # df 데이터의 개수
    n = df.shape[0]
    
    # label의 정답 값(꼭 yes나 no라는 보장이 없으므로)
    label_value = np.unique(df[label])
    
    # df의 label 컬럼 중 첫번째 레이블 수
    label1 = np.sum(df[label] == label_value[0])
    
    # df의 label 컬럼 중 두번째 레이블 수(2번째 까지만 고려, binary classification)
    label2 = n - label1 # 첫번째 레이블 갯수 + 두번째 레이블 갯수 = 전체 데이터 갯수
    
    # 지니 계수 계산
    gini = 1 - (label1 / n) ** 2 - (label2 / n) ** 2
    
    return gini

In [8]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [9]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result=[]
    
    # 입력 df의 attribute 컬럼의 unique 값들을 추출
    unique_value = np.unique(df[attribute])
    
    # combinations를 이용해 원소가 1, unique한 원소 개수보다 1개 작은 숫자까지 조합 생성
    # unique한 갯수가 n이라고 가정하면 1<->n-1, 2<->n-2, ... 꼴로 매칭이 되는 쌍들이 나와야함.(2^(n-1) - 1개)
    for i in range(1, len(unique_value)):
        # unique한 값들을 가지고 1개 또는 2개의 원소를 갖도록 조합 실행
        for att in combinations(unique_value, i):
            result.append(list(att))
    
    return result

In [10]:
get_binary_split(pd_data,'income')

[['high'],
 ['low'],
 ['medium'],
 ['high', 'low'],
 ['high', 'medium'],
 ['low', 'medium']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [11]:
def get_attribute_gini_index(df, attribute, label):
    
    # DataFrame의 obs 갯수
    n = df.shape[0]
    
    # 일단 경우의 수로 split
    split = get_binary_split(df, attribute)
    
    # 결과를 담을 dictionary 선언
    result = {}
    
    # split에 담겨있는 원소 하나씩 수행
    for col in split:
        # dictionary에 담을 이름 선언(2개 이상일시 _로 구분되고 1개일 시 그냥 들어갈 것.)
        col_name = "_".join(col)
        
        # 해당 attribute가 col에 속하는가??
        # 뒤집기 연산 (~)을 하기위해 numpy array 형식으로 지정.
        mask = np.array(list(map(lambda x : x in col, df[attribute])))
        
        # 원하는 DataFrame만 뽑아주기(t = target, r = rest)
        data_t = df.loc[mask, :]
        data_r = df.loc[~mask, :]        

        # gini계수 계산
        gini_t = get_gini(data_t, label)
        gini_r = get_gini(data_r, label)
        
        gini = len(data_t) / n * gini_t + len(data_r) / n * gini_r
        
        # dictionary에 추가
        result[col_name] = gini
    
    return result

In [12]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.44285714285714295,
 'high_low': 0.4583333333333333,
 'high_medium': 0.45,
 'low': 0.45,
 'low_medium': 0.44285714285714295,
 'medium': 0.4583333333333333}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [13]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.44285714285714295)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [15]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
     # 해당 변수 중 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(pd_data, feature, label).items())[1]
    print(f"Minimum Gini index of {feature} : {min_gini:.4f}")


Minimum Gini index of age : 0.3571
Minimum Gini index of income : 0.4429
Minimum Gini index of student : 0.3673
Minimum Gini index of credit_rating : 0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [16]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'middle_aged': 0.35714285714285715,
 'middle_aged_senior': 0.3936507936507937,
 'middle_aged_youth': 0.45714285714285713,
 'senior': 0.45714285714285713,
 'senior_youth': 0.35714285714285715,
 'youth': 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [17]:
#선정한 feature로 데이터프레임 split을 해주세요.

df_1 = pd_data[pd_data['age'] == 'middle_aged']
df_2 = pd_data[pd_data['age'] != 'middle_aged']

In [18]:
df_1.head() #split 결과 확인 

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [19]:
df_2.head() #split 결과 확인

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [21]:
# 각 변수를 대상으로 반복문 수행
# 결과는 소수점 이하 4자리까지 출력

for col in features:
    # 해당 변수 중 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(df_1, col, label).items())[1]
    print(f"Age : Middle_aged, Minimum Gini index of {col} : {min_gini:.4f}")

print("#############################################################")
# Youth, Senior DataFrame
# 각 변수를 대상으로 반복문 수행
for col in features:
    # 해당 변수 중 가장 낮은 gini 계수와 변수 출력
    min_gini = min(get_attribute_gini_index(df_2, col, label).items())[1]
    print(f"Age : Middle_aged, Minimum Gini index of {col} : {min_gini:.4f}")

Age : Middle_aged, Minimum Gini index of income : 0.0000
Age : Middle_aged, Minimum Gini index of student : 0.0000
Age : Middle_aged, Minimum Gini index of credit_rating : 0.0000
#############################################################
Age : Middle_aged, Minimum Gini index of income : 0.3750
Age : Middle_aged, Minimum Gini index of student : 0.3200
Age : Middle_aged, Minimum Gini index of credit_rating : 0.4167


In [22]:
df_1

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [23]:
df_2

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes
10,youth,medium,yes,excellent,yes
13,senior,medium,no,excellent,no


## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [28]:
from math import log2

def getEntropy(df, feature) :
    
    entropy = 0
    feature_uniq = list(df[feature].unique())

    for uniq in feature_uniq:
        entropy += - (sum(df[feature] == uniq) / len(df[feature])) * np.log2((sum(df[feature] == uniq) / len(df[feature])))
    
    return(entropy)

In [29]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706309

In [34]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}

    info_2 = getEntropy(df, feature) # 목표변수에 대한 Entropy 를 구하기
    columns = list(df.loc[:, df.columns != feature]) # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장

    for col in columns:
        info = 0
        for uniq in list(df[col].unique()):
            info +=  (sum(df[col]==uniq) / len(df[feature])) * getEntropy(df[df[col] == uniq], feature)
        result[col] = info_2 - info
    
    return(result)

In [35]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.2467498197744391,
 'credit_rating': 0.04812703040826927,
 'income': 0.029222565658954647,
 'student': 0.15183550136234136}